In [3]:
import cv2
import numpy as np

In [4]:
import matplotlib.pyplot as plt
# 将绘制出的图像直接嵌入notebook单元格中
%matplotlib inline
# 定义可视化图像函数
def look_img(img):
    ''' opencv读入图像格式为BGR，matplotlib可视化格式为RGB，因此需将BGR转RGB '''
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_RGB)
    plt.show()

In [ ]:
# 导入预训练YOLOV3模型
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

In [ ]:
# 导入类别
with open('coco.names', 'r') as f:
    classes = f.read().splitlines()

In [ ]:
classes

In [ ]:
# 导入图像
img = cv2.imread('test.jpg')

In [ ]:
look_img(img)

In [ ]:
img.shape

In [ ]:
# 获取图像宽高
height, width, _ = img.shape

In [ ]:
# 对图像预处理
blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0), swapRB=True, crop=False)

In [ ]:
blob.shape

In [ ]:
net.setInput(blob)

In [ ]:
# 获取网络所有层名字
net.getLayerNames()

In [ ]:
# 获取某一层的权重值
net.getParam('conv_14').shape

In [ ]:
# 获取三个尺度输出层的索引号
net.getUnconnectedOutLayers()

In [5]:
# 获取三个尺度输出层的名称
layersNames = net.getLayerNames()
output_layers_names = [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]
output_layers_names

NameError: name 'net' is not defined

In [ ]:
# 前向推断
prediction = net.forward(output_layers_names)

In [ ]:
# 分析三个尺度输出结果
len(prediction)

In [ ]:
prediction[0].shape

In [ ]:
prediction[1].shape

In [ ]:
prediction[2].shape

In [ ]:
# 查看第二个尺度，索引为99的框的85维向量
prediction[1][99].shape

In [ ]:
prediction[1][99]

In [ ]:
# 从三个尺度输出结果中解析所有预测框信息

# 存放预测框坐标
boxes = []

# 存放置信度
objectness = []

# 存放预测框类别索引号
class_ids = []

# 存放预测框类别名称
class_names = []

In [ ]:
for scale in prediction:  # 遍历三种尺度
    for bbox in scale:    # 遍历每个预测框
        obj = bbox[4]     # 获取该预测框的confidence(objectness)
        class_scores = bbox[5:]  # 获取该预测框在COCO数据集80个类别的概率
        class_id = np.argmax(class_scores)  # 获取概率最高类别的索引号
        class_name = classes[class_id]  # 获取概率最高类别的名称
        class_prob = class_scores[class_id]  # 获取概率最高类别的概率